<a href="https://colab.research.google.com/github/nishantkapps/aiml-projects/blob/main/Shell_EV_Charging_Stations_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns=None

In [ ]:
hist = pd.read_csv('/content/drive/MyDrive/datasets/shell/Demand_History_And_Preds.csv')
ev = pd.read_csv('/content/drive/MyDrive/datasets/shell/existing_EV_infrastructure_2018.csv')
sub = pd.read_csv('/content/drive/MyDrive/datasets/shell/sample_submission.csv')
ev['fsup']=ev['existing_num_FCS']*400
ev['ssup']=ev['existing_num_SCS']*200

In [ ]:
DEM2018 = np.array(hist.iloc[:,[0,1,2,11]])
DEM2019 = np.array(hist.iloc[:,[0,1,2,12]])
DEM2020 = np.array(hist.iloc[:,[0,1,2,13]])
SUP2018 = np.array(ev)

In [ ]:
#SUP2018

In [ ]:
#plt.figure(figsize=(100,100))
#plt.scatter(x=DEM2018[:,1],y=DEM2018[:,2],s=DEM2018[:,3])
#plt.scatter(x=SUP2018[:,1],y=SUP2018[:,2],s=SUP2018[:,3])
#plt.show()

In [ ]:
#plt.figure(figsize=(100,100))
#plt.scatter(x=DEM2020[:,1],y=DEM2020[:,2],s=DEM2020[:,3])
#plt.scatter(x=SUP2018[:,1],y=SUP2018[:,2],s=SUP2018[:,3])
#plt.show()

In [ ]:
# Merge columns to create datasets for each year with demand/supply points, coordinates, demand/supply volumes
Dist2018 = pd.merge(pd.DataFrame(DEM2018,columns=['dp','x1','y1','demand']),pd.DataFrame(SUP2018,columns=['sp','x2','y2','tot','scs','fcs','fsup','ssup']),how='cross')
Dist2019 = pd.merge(pd.DataFrame(DEM2019,columns=['dp','x1','y1','demand']),pd.DataFrame(SUP2018,columns=['sp','x2','y2','tot','scs','fcs','fsup','ssup']),how='cross')
Dist2020 = pd.merge(pd.DataFrame(DEM2020,columns=['dp','x1','y1','demand']),pd.DataFrame(SUP2018,columns=['sp','x2','y2','tot','scs','fcs','fsup','ssup']),how='cross')

In [ ]:
# Calculate distance between demand and supply points
Dist2018['distance'] = abs(Dist2018['x1']-Dist2018['x2'])+abs(Dist2018['y1']-Dist2018['y2'])
Dist2019['distance'] = abs(Dist2019['x1']-Dist2019['x2'])+abs(Dist2019['y1']-Dist2019['y2'])
Dist2020['distance'] = abs(Dist2020['x1']-Dist2020['x2'])+abs(Dist2020['y1']-Dist2020['y2'])

In [ ]:
# Sort by distance ascending ( min value is the first row )
Dist2018=Dist2018.sort_values(by=['dp','distance'], ascending=[True,True])
Dist2019=Dist2019.sort_values(by=['dp','distance'], ascending=[True,True])
Dist2020=Dist2020.sort_values(by=['dp','distance'], ascending=[True,True])

In [ ]:
# Add an index to identify minimum to maximum value in each dataframe
Dist2018['sortindex']=Dist2018.groupby('dp').cumcount()+1
Dist2019['sortindex']=Dist2018.groupby('dp').cumcount()+1
Dist2020['sortindex']=Dist2018.groupby('dp').cumcount()+1

In [ ]:
# Define the maximum no of supply points that a demand point can use
# We are making an assumption here that people will go only to their XX nearest supply points for recharge
# Alternative is to assume the demand is fulfilled from all supply points based on distance
# max_no_of_sp_used_by_dp = 20
Dist2018['dsij']=(Dist2018['demand']/20)*(10/(2**(Dist2018['sortindex']-1)))
Dist2019['dsij']=(Dist2019['demand']/20)*(10/(2**(Dist2019['sortindex']-1)))
Dist2020['dsij']=(Dist2020['demand']/20)*(10/(2**(Dist2020['sortindex']-1)))
Dist2018['dsij'].values[Dist2018['dsij']<1e-3]=0
Dist2019['dsij'].values[Dist2019['dsij']<1e-3]=0
Dist2020['dsij'].values[Dist2020['dsij']<1e-3]=0
Dist2018['dsij'].values[Dist2018['dsij']==np.inf]=0
Dist2019['dsij'].values[Dist2019['dsij']==np.inf]=0
Dist2020['dsij'].values[Dist2020['dsij']==np.inf]=0
ds18 = pd.pivot_table(data=Dist2018, index=['sp','tot','fcs','scs','fsup','ssup','demand'], values='dsij', aggfunc='sum').reset_index()
ds19 = pd.pivot_table(data=Dist2019, index=['sp','tot','fcs','scs','fsup','ssup','demand'], values='dsij', aggfunc='sum').reset_index()
ds20 = pd.pivot_table(data=Dist2020, index=['sp','tot','fcs','scs','fsup','ssup','demand'], values='dsij', aggfunc='sum').reset_index()

In [ ]:
Dist2018['supply']=Dist2019['fsup']+Dist2019['ssup']
Dist2019['supply']=Dist2019['fsup']+Dist2019['ssup']
Dist2020['supply']=Dist2019['fsup']+Dist2019['ssup']

In [ ]:
Dist2018.fillna(0,inplace=True)
Dist2019.fillna(0,inplace=True)
Dist2020.fillna(0,inplace=True)

In [ ]:
Dist2019.loc[(Dist2019['sp']==10)&(Dist2019['dsij']!=0)].groupby(['sp']).agg({'dsij':'sum'}).sort_values(['sp','dsij'],ascending=[True,False])

,dsij
sp,
10.0,6625.321139


In [ ]:
Dist2019.groupby(['sp','tot']).agg({'dsij':'sum'})

,,dsij
sp,tot,
0.0,23.0,3100.702629
1.0,27.0,4074.502858
2.0,31.0,2093.767119
3.0,26.0,3305.336155
4.0,32.0,2461.483545
...,...,...
95.0,24.0,4101.698322
96.0,32.0,7365.077472
97.0,32.0,6662.926552


In [ ]:
Dist2019.loc[(Dist2019['sp']==10)&(Dist2019['dsij']!=0)][['dp','dsij','distance']].sort_values('distance',ascending=False)

,dp,dsij,distance
2510,25.0,0.001251,32.657319
2610,26.0,0.001334,31.657319
8910,89.0,0.001584,31.657319
2710,27.0,0.001299,30.657319
9010,90.0,0.001536,30.657319
...,...,...,...
43710,437.0,22.885052,1.342681
50010,500.0,27.464946,1.342681
37210,372.0,37.158444,1.220443
43510,435.0,38.020640,0.779557


In [ ]:
ds19['evdg']=ds19['dsij']-ds19['fsup']-ds19['ssup']
ds19['evdg'].values[ds19['evdg']<0]=0
ds19['fcs2']=(ds19['evdg']//400)
ds19.loc[ds19['evdg']>0,'scs2']=((ds19['evdg']-(ds19['fcs2']*400))//200)+1
ds19.loc[ds19['evdg']==0,'scs2']=0
ds19['fcs_new']=ds19['fcs']+ds19['fcs2']
ds19['scs_new']=ds19['scs']+ds19['scs2']

# Resolving for Constraint 3
ds19['rempre']=ds19['tot']-ds19['fcs_new']-ds19['scs_new']
ds19.loc[ds19['rempre']<0,'fcs_new']=ds19['tot']-ds19['scs']
ds19.loc[ds19['rempre']<0,'scs_new']=ds19['scs']
ds19['rempost']=ds19['tot']-ds19['fcs_new']-ds19['scs_new']

# Resolving for Constraint 4
ds19['fcs_low_pre']=ds19['fcs_new']-ds19['fcs']
ds19['scs_low_pre']=ds19['scs_new']-ds19['scs']
ds19.loc[ds19['fcs_low_pre']<0,'fcs_new']=ds19['fcs']
ds19.loc[ds19['scs_low_pre']<0,'scs_new']=ds19['scs']
ds19['fcs_low_post']=ds19['fcs_new']-ds19['fcs']
ds19['scs_low_post']=ds19['scs_new']-ds19['scs']

In [ ]:
ds19

,sp,tot,fcs,scs,fsup,ssup,demand,dsij,evdg,fcs2,scs2,fcs_new,scs_new,rempre,rempost,fcs_low_pre,scs_low_pre,fcs_low_post,scs_low_post,year
0,0.0,23.0,3.0,5.0,1200.0,1000.0,0.000000,0.000000,0.0,0.0,0.0,3.0,5.0,15.0,15.0,0.0,0.0,0.0,0.0,2019
1,0.0,23.0,3.0,5.0,1200.0,1000.0,0.190609,0.005957,0.0,0.0,0.0,3.0,5.0,15.0,15.0,0.0,0.0,0.0,0.0,2019
2,0.0,23.0,3.0,5.0,1200.0,1000.0,0.355845,0.000000,0.0,0.0,0.0,3.0,5.0,15.0,15.0,0.0,0.0,0.0,0.0,2019
3,0.0,23.0,3.0,5.0,1200.0,1000.0,0.808687,0.000000,0.0,0.0,0.0,3.0,5.0,15.0,15.0,0.0,0.0,0.0,0.0,2019
4,0.0,23.0,3.0,5.0,1200.0,1000.0,0.813278,0.000000,0.0,0.0,0.0,3.0,5.0,15.0,15.0,0.0,0.0,0.0,0.0,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406695,99.0,24.0,11.0,2.0,4400.0,400.0,345.993522,0.000000,0.0,0.0,0.0,11.0,2.0,11.0,11.0,0.0,0.0,0.0,0.0,2019
406696,99.0,24.0,11.0,2.0,4400.0,400.0,347.647178,0.000000,0.0,0.0,0.0,11.0,2.0,11.0,11.0,0.0,0.0,0.0,0.0,2019
406697,99.0,24.0,11.0,2.0,4400.0,400.0,352.793133,0.000000,0.0,0.0,0.0,11.0,2.0,11.0,11.0,0.0,0.0,0.0,0.0,2019
406698,99.0,24.0,11.0,2.0,4400.0,400.0,361.854336,0.000000,0.0,0.0,0.0,11.0,2.0,11.0,11.0,0.0,0.0,0.0,0.0,2019


In [ ]:
# Resolving for Constraint 5
# ds19.loc[ds19['fcs_new']!=(ds19['fcs']+ds19['fcs2'])]

In [ ]:
ds20['fcs19']=ds19['fcs_new']
ds20['scs19']=ds19['scs_new']
ds20['evdg']=ds20['dsij']-ds20['fcs19']*400-ds20['scs19']*200
ds20['evdg'].values[ds20['evdg']<0]=0
ds20['fcs2']=(ds20['evdg']//400)
ds20.loc[ds20['evdg']>0,'scs2']=((ds20['evdg']-(ds20['fcs2']*400))//200)+1
ds20.loc[ds20['evdg']==0,'scs2']=0
ds20['fcs_new']=ds20['fcs19']+ds20['fcs2']
ds20['scs_new']=ds20['scs19']+ds20['scs2']

# Resolving for Constraint 3
ds20['rempre']=ds20['tot']-ds20['fcs_new']-ds20['scs_new']
ds20.loc[ds20['rempre']<0,'fcs_new']=ds20['tot']-ds20['scs19']
ds20.loc[ds20['rempre']<0,'scs_new']=ds20['scs19']
ds20['rempost']=ds20['tot']-ds20['fcs_new']-ds20['scs_new']

# Resolving for Constraint 4
ds20['fcs_low_pre']=ds20['fcs_new']-ds20['fcs19']
ds20['scs_low_pre']=ds20['scs_new']-ds20['scs19']
ds20.loc[ds20['fcs_low_pre']<0,'fcs_new']=ds20['fcs19']
ds20.loc[ds20['scs_low_pre']<0,'scs_new']=ds20['scs19']
ds20['fcs_low_post']=ds20['fcs_new']-ds20['fcs19']
ds20['scs_low_post']=ds20['scs_new']-ds20['scs19']

# Resolving for Constraint 5

In [ ]:
ds20.loc[ds20['sp'].isin([10,17,35,37,66,74,76,90,92])]

,sp,tot,fcs,scs,fsup,ssup,demand,dsij,fcs19,scs19,evdg,fcs2,scs2,fcs_new,scs_new,rempre,rempost,fcs_low_pre,scs_low_pre,fcs_low_post,scs_low_post
40670,10.0,14.0,0.0,6.0,0.0,1200.0,0.000000,0.000000,0.0,6.0,0.0,0.0,0.0,0.0,6.0,8.0,8.0,0.0,0.0,0.0,0.0
40671,10.0,14.0,0.0,6.0,0.0,1200.0,0.214016,0.107008,0.0,6.0,0.0,0.0,0.0,0.0,6.0,8.0,8.0,0.0,0.0,0.0,0.0
40672,10.0,14.0,0.0,6.0,0.0,1200.0,0.412760,0.000000,0.0,6.0,0.0,0.0,0.0,0.0,6.0,8.0,8.0,0.0,0.0,0.0,0.0
40673,10.0,14.0,0.0,6.0,0.0,1200.0,0.850569,0.000000,0.0,6.0,0.0,0.0,0.0,0.0,6.0,8.0,8.0,0.0,0.0,0.0,0.0
40674,10.0,14.0,0.0,6.0,0.0,1200.0,0.911366,0.000000,0.0,6.0,0.0,0.0,0.0,0.0,6.0,8.0,8.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378226,92.0,19.0,3.0,6.0,1200.0,1200.0,395.171031,0.000000,3.0,6.0,0.0,0.0,0.0,3.0,6.0,10.0,10.0,0.0,0.0,0.0,0.0
378227,92.0,19.0,3.0,6.0,1200.0,1200.0,395.508711,0.000000,3.0,6.0,0.0,0.0,0.0,3.0,6.0,10.0,10.0,0.0,0.0,0.0,0.0
378228,92.0,19.0,3.0,6.0,1200.0,1200.0,404.323735,0.000000,3.0,6.0,0.0,0.0,0.0,3.0,6.0,10.0,10.0,0.0,0.0,0.0,0.0
378229,92.0,19.0,3.0,6.0,1200.0,1200.0,405.195781,0.000000,3.0,6.0,0.0,0.0,0.0,3.0,6.0,10.0,10.0,0.0,0.0,0.0,0.0


In [ ]:
#a=ds20.loc[ds20['sp']==50]['evdg']
#b=a//400
#c=((a-b*400)//200)+1

In [ ]:
#print(a)
#print(b)
#print(c)

In [ ]:
#ds19.loc[ds19['sp']==50]

In [ ]:
#ds20.loc[ds20['sp']==50]

In [ ]:
ds19['year']='2019'
sub19scs = ds19[['year','sp','scs_new']]
sub19fcs = ds19[['year','sp','fcs_new']]
sub19fcs['data_type'] = 'FCS'
sub19fcs['dp'] = ''
sub19scs['data_type'] = 'SCS'
sub19scs['dp'] = ''
sub19scs.rename(columns={'scs_new':'value'},inplace=True)
sub19fcs.rename(columns={'fcs_new':'value'},inplace=True)
sub2019 = pd.concat([sub19fcs,sub19scs],axis=0)
sub2019.fillna('',inplace=True)
sub2019 = sub2019[['year','data_type','dp','sp','value']]

In [ ]:
#sub19scs

In [ ]:
ds20['year']='2020'
sub20scs = ds20[['year','sp','scs_new']]
sub20fcs = ds20[['year','sp','fcs_new']]
sub20fcs['data_type'] = 'FCS'
sub20fcs['dp'] = ''
sub20scs['data_type'] = 'SCS'
sub20scs['dp'] = ''
sub20scs.rename(columns={'scs_new':'value'},inplace=True)
sub20fcs.rename(columns={'fcs_new':'value'},inplace=True)
sub2020 = pd.concat([sub20fcs,sub20scs],axis=0)
sub2020.fillna('',inplace=True)
sub2020 = sub2020[['year','data_type','dp','sp','value']]

In [ ]:
dsij2019 = Dist2019[['dp','sp','dsij']]
dsij2020 = Dist2020[['dp','sp','dsij']]

In [ ]:
dsij2019['year']='2019'
dsij2019['data_type']='DS'
dsij2020['year']='2020'
dsij2020['data_type']='DS'
subd2019 = dsij2019[['year','data_type','dp','sp','dsij']]
subd2020 = dsij2020[['year','data_type','dp','sp','dsij']]

In [ ]:
# Resolving for Constraint 5
#subd2019.groupby('sp').sum()

In [ ]:
subd2019.rename(columns={'dsij':'value'},inplace=True)
subd2020.rename(columns={'dsij':'value'},inplace=True)

In [ ]:
# Resolving for Constraint 1
subd2019.fillna(0,inplace=True)
subd2020.fillna(0,inplace=True)

In [ ]:
submission = pd.concat([sub2019,sub2020,subd2019,subd2020],axis=0)

In [ ]:
submission.rename(columns={'dp':'demand_point_index','sp':'supply_point_index'},inplace=True)

In [ ]:
submission.to_csv('submission18.csv', encoding='utf-8', index=False)

In [ ]:
#Dist2019[Dist2019['dp']==0.0].sort_values('dsij',ascending=False)

In [ ]:
#subd2019.isnull().sum()

In [ ]:
#x=[]
#a=13.1195721
#b=(a/(2**100-1))
#for i in range(100):
#  x.append((2**i)*b)
#np.sum(x)